<a href="https://colab.research.google.com/github/dathere/notebooks/blob/master/NYC_borough_covid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INTRODUCTION

Currently, the widely used [JHU COVID19 dashboard](https://coronavirus.jhu.edu/map.html) and even the [NYTimes COVID19 data](https://github.com/nytimes/covid-19-data) only have data at the city level for NYC, not down to the borough level.

Fortunately, [USAFacts has their own curated COVID19 opendata feed](https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/) with borough level, time-series data.  

This notebook wrangles this data, along with other datasets (NYC DOHMH, US Census, NYU and community-curated data) and creates several plotly visualizations for all 5 boroughs.

First, let's install plotly & pandas...

In [87]:
!pip install plotly==4.6.0
!pip install pandas

# USA Facts Data

## Process USAFacts Confirmed Data


In [88]:
import pandas as pd
url = "https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_confirmed_usafacts.csv"
confirmed_df=pd.read_csv(url)
confirmed_df

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4,6,6,6,6,7,7,10,10,12,12,12,12,12,12,17,17,19,19,19,23,25,25,25,25
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,1,1,1,1,2,2,3,3,4,4,5,5,10,15,18,19,23,25,28,29,34,38,42,49,59,59,66,71,78,87,98,102,103,109
3,1005,Barbour County,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,3,3,3,7,9,10,10,9,11,13,14,15,18
4,1007,Bibb County,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3,3,4,4,4,7,7,8,9,11,11,13,16,17,17,19,23,23,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3190,56037,Sweetwater County,WY,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,2,2,3,3,3,4,5,5,6,6,6,7,7,7,9,9,10,10,10,10
3191,56039,Teton County,WY,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1,2,2,2,2,3,6,8,12,14,14,17,23,26,29,32,36,39,40,44,45,50,53,53,56,56,57,59,59,61,62
3192,56041,Uinta County,WY,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,2,2,3,3,3,4,4,4,4,4,4,4,4,6,6
3193,56043,Washakie County,WY,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,2,2,2,4,4,5,5,5,5,5,5,5,5,5,5,5,5


After fetching the data, let's filter it just for NYC boroughs, remove some extraneous columns, and then transpose the data for graphing.

In [89]:
nyc_counties_df = confirmed_df.loc[(confirmed_df['State'] == 'NY') & (confirmed_df['countyFIPS'].isin([36005, 36061, 36081, 36085, 36047]))]

nyc_counties_df=nyc_counties_df.set_index('County Name')
nyc_counties_df=nyc_counties_df.drop(columns=['countyFIPS', 'State', 'stateFIPS'])

nyc_counties_df = nyc_counties_df.transpose()
nyc_counties_df = nyc_counties_df.rename_axis('Date')

nyc_counties_df.index =pd.to_datetime(nyc_counties_df.index)
nyc_counties_df

County Name,Bronx County,Kings County,New York County,Queens County,Richmond County
Date,,,,,
2020-01-22,0,0,0,0,0
2020-01-23,0,0,0,0,0
2020-01-24,0,0,0,0,0
2020-01-25,0,0,0,0,0
2020-01-26,0,0,0,0,0
...,...,...,...,...,...
2020-04-14,23266,29306,14880,35053,7960
2020-04-15,24653,31969,16617,36765,8298
2020-04-16,25932,33521,17091,37918,8684


Manhattan ("New York County") had the first COVID19 case on Mar 2, let's filter the dataframe so we start with the first COVID19 confirmed case in Manhattan.

In [90]:
nyc_counties_df['Total'] = nyc_counties_df['Bronx County'] + nyc_counties_df['Kings County'] + nyc_counties_df['New York County'] + nyc_counties_df['Queens County'] + nyc_counties_df['Richmond County']

df = nyc_counties_df
df = df.loc['2020-03-02':]

df

County Name,Bronx County,Kings County,New York County,Queens County,Richmond County,Total
Date,,,,,,
2020-03-02,0,0,1,0,0,1
2020-03-03,0,0,1,0,0,1
2020-03-04,0,0,1,0,0,1
2020-03-05,0,1,3,0,0,4
2020-03-06,0,1,4,0,0,5
2020-03-07,0,3,8,1,0,12
2020-03-08,1,3,8,1,0,13
2020-03-09,2,4,11,2,1,20
2020-03-10,3,10,17,4,2,36


Create the plotly graph for confirmed cases.

In [91]:
import plotly.graph_objects as go

def prepFigure(figure, title):
    figure.update_xaxes(tickformat='%a<br>%b %d',
        tick0 = '2020-03-22',
        dtick = 7 * 24 * 3600000,
        rangeselector=dict(
            buttons=list([
                dict(count=7, label="1w", step="day", stepmode="backward"),
                dict(count=14, label="2w", step="day", stepmode="backward"),
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(step="all")
            ])
        )
    )
    figure.update_layout(title="<b>" + title + "</b>", hovermode="x")
    return figure

fig2 = go.Figure()
fig2.add_scatter(x=df.index, y=df['Bronx County'], mode='lines', name='Bronx')
fig2.add_scatter(x=df.index, y=df['Kings County'], mode='lines',name='Brooklyn')
fig2.add_scatter(x=df.index, y=df['Queens County'], mode='lines', name='Queens')
fig2.add_scatter(x=df.index, y=df['New York County'], mode='lines', name='Manhattan')
fig2.add_scatter(x=df.index, y=df['Richmond County'], mode='lines', name='Staten Island')
fig2.add_scatter(x=df.index, y=df['Total'], mode='lines+markers', name='NYC')

prepFigure(fig2, "NYC COVID19 Confirmed Cases")

Connect to Google Drive and save the graph HTML there.

In [92]:
from google.colab import drive
drive.mount('/content/drive')

fig2.write_html('/content/drive/My Drive/publicweb/nyc-boroughs-covid19.html', config={"showLink": True,
  "plotlyServerURL": "https://chart-studio.plotly.com"} , include_plotlyjs="cdn")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [93]:
fig2.update_layout(title="<b>NYC COVID19 Confirmed Cases - log scale</b>", yaxis_type="log")

In [0]:
fig2.write_html('/content/drive/My Drive/publicweb/nyc-boroughs-covid19-log.html', config={"showLink": True,
  "plotlyServerURL": "https://chart-studio.plotly.com"} , include_plotlyjs="cdn")

## Process USAFacts Deaths Data

Use the same process for COVID19 deaths...

In [95]:
url = "https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_deaths_usafacts.csv"
deaths_df = pd.read_csv(url)
deaths_df

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,1005,Barbour County,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1007,Bibb County,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3190,56037,Sweetwater County,WY,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3191,56039,Teton County,WY,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3192,56041,Uinta County,WY,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3193,56043,Washakie County,WY,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [96]:
nyc_counties_deaths_df = deaths_df.loc[(deaths_df['State'] == 'NY') & (deaths_df['countyFIPS'].isin([36005, 36061, 36081, 36085, 36047]))]

nyc_counties_deaths_df=nyc_counties_deaths_df.set_index('County Name')
nyc_counties_deaths_df=nyc_counties_deaths_df.drop(columns=['countyFIPS', 'State', 'stateFIPS'])

nyc_counties_deaths_df = nyc_counties_deaths_df.transpose()
nyc_counties_deaths_df = nyc_counties_deaths_df.rename_axis('Date')
nyc_counties_deaths_df.index =  pd.to_datetime(nyc_counties_deaths_df.index)

nyc_counties_deaths_df

County Name,Bronx County,Kings County,New York County,Queens County,Richmond County
Date,,,,,
2020-01-22,0,0,0,0,0
2020-01-23,0,0,0,0,0
2020-01-24,0,0,0,0,0
2020-01-25,0,0,0,0,0
2020-01-26,0,0,0,0,0
...,...,...,...,...,...
2020-04-14,1656,2498,963,2421,367
2020-04-15,1771,2672,1019,2604,389
2020-04-16,1845,2811,1070,2754,413


The first death was on Mar 14 in Brooklyn...


In [97]:
nyc_counties_deaths_df['Total'] = nyc_counties_deaths_df['Bronx County'] + nyc_counties_deaths_df['Kings County'] + nyc_counties_deaths_df['New York County'] + nyc_counties_deaths_df['Queens County'] + nyc_counties_deaths_df['Richmond County']

df2 = nyc_counties_deaths_df

df2 = nyc_counties_deaths_df.loc['2020-03-14':]
df2

County Name,Bronx County,Kings County,New York County,Queens County,Richmond County,Total
Date,,,,,,
2020-03-14,0,1,0,0,0,1
2020-03-15,0,1,4,0,0,5
2020-03-16,1,1,0,0,0,2
2020-03-17,1,1,0,0,0,2
2020-03-18,1,1,0,0,0,2
2020-03-19,1,1,0,0,0,2
2020-03-20,1,1,0,0,0,2
2020-03-21,0,1,0,0,0,1
2020-03-22,14,14,10,21,4,63


In [98]:
fig2 = go.Figure()
fig2.add_scatter(x=df2.index, y=df2['Bronx County'], mode='lines', name='Bronx')
fig2.add_scatter(x=df2.index, y=df2['Kings County'], mode='lines',name='Brooklyn')
fig2.add_scatter(x=df2.index, y=df2['Queens County'], mode='lines', name='Queens')
fig2.add_scatter(x=df2.index, y=df2['New York County'], mode='lines', name='Manhattan')
fig2.add_scatter(x=df2.index, y=df2['Richmond County'], mode='lines', name='Staten Island')
fig2.add_scatter(x=df2.index, y=df2['Total'], mode='lines+markers', name='NYC')

prepFigure(fig2,title="NYC COVID19 Deaths")

In [0]:
fig2.write_html('/content/drive/My Drive/publicweb/nyc-boroughs-covid19-deaths.html', config={"showLink": True,
  "plotlyServerURL": "https://chart-studio.plotly.com"} , include_plotlyjs="cdn")

In [100]:
fig2.update_layout(title="<b>NYC COVID19 Deaths - log scale</b>", yaxis_type="log")

In [0]:
fig2.write_html('/content/drive/My Drive/publicweb/nyc-boroughs-covid19-deaths-log.html', config={"showLink": True,
  "plotlyServerURL": "https://chart-studio.plotly.com"} , include_plotlyjs="cdn")

## Compute Case Fatality Rate

Now let's compute the case fatality rate...

In [102]:
cfr_df = nyc_counties_deaths_df / nyc_counties_df

cfr_df 

County Name,Bronx County,Kings County,New York County,Queens County,Richmond County,Total
Date,,,,,,
2020-01-22,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-23,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-24,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-25,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-26,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2020-04-14,0.071177,0.085239,0.064718,0.069067,0.046106,0.071561
2020-04-15,0.071837,0.083581,0.061323,0.070828,0.046879,0.071470
2020-04-16,0.071148,0.083858,0.062606,0.072630,0.047559,0.072215


In [103]:
# Let's start the chart on Mar 14
df2 = cfr_df.loc['2020-03-14':]
df2

County Name,Bronx County,Kings County,New York County,Queens County,Richmond County,Total
Date,,,,,,
2020-03-14,0.000000,0.029412,0.000000,0.000000,0.000000,0.005464
2020-03-15,0.000000,0.018868,0.055556,0.000000,0.000000,0.020833
2020-03-16,0.029412,0.016129,0.000000,0.000000,0.000000,0.005814
2020-03-17,0.010417,0.006369,0.000000,0.000000,0.000000,0.002457
2020-03-18,0.003300,0.001980,0.000000,0.000000,0.000000,0.000996
2020-03-19,0.002016,0.000837,0.000000,0.000000,0.000000,0.000506
2020-03-20,0.001499,0.000659,0.000000,0.000000,0.000000,0.000389
2020-03-21,0.000000,0.000403,0.000000,0.000000,0.000000,0.000123
2020-03-22,0.009922,0.004900,0.004826,0.007735,0.006745,0.006530


In [104]:
fig2 = go.Figure()
fig2.add_scatter(x=df2.index, y=df2['Bronx County'], mode='lines', name='Bronx')
fig2.add_scatter(x=df2.index, y=df2['Kings County'], mode='lines',name='Brooklyn')
fig2.add_scatter(x=df2.index, y=df2['Queens County'], mode='lines', name='Queens')
fig2.add_scatter(x=df2.index, y=df2['New York County'], mode='lines', name='Manhattan')
fig2.add_scatter(x=df2.index, y=df2['Richmond County'], mode='lines', name='Staten Island')
fig2.add_scatter(x=df2.index, y=df2['Total'], mode='lines+markers', name='NYC')

prepFigure(fig2, title='NYC Case Fatality Rate')
fig2.update_layout(yaxis_title="Percent", yaxis_tickformat=".2%",
                   xaxis_title="""
<b>What do we know about the risk of dying from COVID-19?</b> 
<i>by Hannah Ritchie and Max Roser</i>,
Mar 25 2020 - <a href src='https://ourworldindata.org/covid-mortality-risk'>https://ourworldindata.org/covid-mortality-risk</a>
"""
)

fig2.update_layout(annotations=[
            go.layout.Annotation(
                text="""
Note that this is a very simplistic<br>
calculation of Case Fatality Rate (CFR),<br>
just dividing the number of deaths by the<br>
number of confirmed cases.<br><br>
It is tracked here nonetheless, for benchmarking<br>purposes.<br><br>
For a detailed explanation about the challenges<br>
of computing true CFR, the article linked below<br>
from OurWorldinData is highly recommended.
""",
                align='left',
                showarrow=False,
                xref='paper',
                yref='paper',
                x=0.12,
                y=0.95,
                bordercolor='black',
                borderwidth=1
            )
        ] )

In [0]:
fig2.write_html('/content/drive/My Drive/publicweb/nyc-boroughs-covid19-cfr.html', config={"showLink": True,
  "plotlyServerURL": "https://chart-studio.plotly.com"} , include_plotlyjs="cdn")

## Derive Daily Counts

Now, let's graph the Daily incident counts from the cumulative data...

In [106]:
def dailydata(dfcounty):
    dfcountydaily=dfcounty.diff(axis=0)#.fillna(0)
    return dfcountydaily

DailyCases_df=dailydata(nyc_counties_df)
DailyDeaths_df=dailydata(nyc_counties_deaths_df)

DailyCases_df = DailyCases_df.loc['2020-03-02':]

# compute 7-day exponential moving average
DailyCases_df['EMA'] = DailyCases_df['Total'].ewm(span=7).mean()

DailyCases_df

County Name,Bronx County,Kings County,New York County,Queens County,Richmond County,Total,EMA
Date,,,,,,,
2020-03-02,0.0,0.0,1.0,0.0,0.0,1.0,1.000000
2020-03-03,0.0,0.0,0.0,0.0,0.0,0.0,0.428571
2020-03-04,0.0,0.0,0.0,0.0,0.0,0.0,0.243243
2020-03-05,0.0,1.0,2.0,0.0,0.0,3.0,1.251429
2020-03-06,0.0,0.0,1.0,0.0,0.0,1.0,1.169014
2020-03-07,0.0,2.0,4.0,1.0,0.0,7.0,2.942382
2020-03-08,1.0,0.0,0.0,0.0,0.0,1.0,2.381982
2020-03-09,1.0,1.0,3.0,1.0,1.0,7.0,3.664926
2020-03-10,1.0,6.0,6.0,2.0,1.0,16.0,6.999035


In [107]:
df2 = DailyCases_df

fig2 = go.Figure()
fig2.add_scatter(x=df2.index,y=df2['Bronx County'], mode='lines', name='Bronx')
fig2.add_scatter(x=df2.index,y=df2['Kings County'], mode='lines', name='Brooklyn')
fig2.add_scatter(x=df2.index,y=df2['New York County'], mode='lines', name='Manhattan')
fig2.add_scatter(x=df2.index,y=df2['Queens County'], mode='lines', name='Queens')
fig2.add_scatter(x=df2.index,y=df2['Richmond County'], mode='lines', name='Staten Island')
fig2.add_scatter(x=df2.index,y=df2['Total'], mode='lines+markers', name='NYC')

fig2.add_scatter(x=df2.index, y=df2['EMA'], mode='lines', line=dict(color='royalblue', width=4, dash='dot'), name='7 day EMA')

prepFigure(fig2, title="NYC Daily New Cases")

In [0]:
fig2.write_html('/content/drive/My Drive/publicweb/nyc-covid19-dailycases.html', config={"showLink": True,
  "plotlyServerURL": "https://chart-studio.plotly.com"} , include_plotlyjs="cdn")

In [109]:
import numpy as np

DailyDeaths_df = DailyDeaths_df.loc['2020-03-14':]

DailyDeaths_df.replace(0.0,np.nan, inplace=True)
# compute 7-day exponential moving average
DailyDeaths_df['EMA'] = DailyDeaths_df['Total'].ewm(span=7).mean()

#temp_df = DailyDeaths_df.replace(0.0,np.nan)

DailyDeaths_df

County Name,Bronx County,Kings County,New York County,Queens County,Richmond County,Total,EMA
Date,,,,,,,
2020-03-14,NaN,1.0,NaN,NaN,NaN,1.0,1.000000
2020-03-15,NaN,NaN,4.0,NaN,NaN,4.0,2.714286
2020-03-16,1.0,NaN,-4.0,NaN,NaN,-3.0,0.243243
2020-03-17,NaN,NaN,NaN,NaN,NaN,NaN,0.243243
2020-03-18,NaN,NaN,NaN,NaN,NaN,NaN,0.243243
2020-03-19,NaN,NaN,NaN,NaN,NaN,NaN,0.243243
2020-03-20,NaN,NaN,NaN,NaN,NaN,NaN,0.243243
2020-03-21,-1.0,NaN,NaN,NaN,NaN,-1.0,-0.559488
2020-03-22,14.0,13.0,10.0,21.0,4.0,62.0,28.382131


In [110]:
df2 = DailyDeaths_df
#df2 = temp_df
fig2 = go.Figure()
fig2.add_scatter(x=df2.index,y=df2['Bronx County'], mode='lines', name='Bronx')
fig2.add_scatter(x=df2.index,y=df2['Kings County'], mode='lines', name='Brooklyn')
fig2.add_scatter(x=df2.index,y=df2['New York County'], mode='lines', name='Manhattan')
fig2.add_scatter(x=df2.index,y=df2['Queens County'], mode='lines', name='Queens')
fig2.add_scatter(x=df2.index,y=df2['Richmond County'], mode='lines', name='Staten Island')
fig2.add_scatter(x=df2.index,y=df2['Total'], mode='lines+markers', name='NYC')

fig2.add_scatter(x=df2.index, y=df2['EMA'], mode='lines', line=dict(color='royalblue', width=4, dash='dot'), name='7 day EMA')
prepFigure(fig2, title="NYC Daily Deaths")

In [0]:
fig2.write_html('/content/drive/My Drive/publicweb/nyc-covid19-dailydeaths.html', config={"showLink": True,
  "plotlyServerURL": "https://chart-studio.plotly.com"} , include_plotlyjs="cdn")

## Compute Per Capita incidents

USAFacts also conveniently provided a feed for the latest county population data from the US Census.

In [112]:
url = "https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_county_population_usafacts.csv"

county_population_df = pd.read_csv(url)

nyc_counties_pop_df = county_population_df.loc[(county_population_df['State'] == 'NY') & (county_population_df['County Name'].isin(
    ['Bronx County', 'Queens County', 'New York County', 'Kings County', 'Richmond County']))]

nyc_counties_pop_df=nyc_counties_pop_df.drop(columns=['countyFIPS', 'State'])

nyc_counties_pop_df=nyc_counties_pop_df.set_index('County Name')
Total = nyc_counties_pop_df.sum()['population']
nyc_counties_pop_df.loc['Total'] = [Total]

nyc_counties_pop_df

,population
County Name,
Bronx County,1418207
Kings County,2559903
New York County,1628706
Queens County,2253858
Richmond County,476143
Total,8336817


In [113]:
df = nyc_counties_df
df = df.loc['2020-03-02':]

cases_per_capita_df = df.divide(nyc_counties_pop_df['population'], axis=1)*100000

cases_per_capita_df

County Name,Bronx County,Kings County,New York County,Queens County,Richmond County,Total
Date,,,,,,
2020-03-02,0.000000,0.000000,0.061398,0.000000,0.000000,0.011995
2020-03-03,0.000000,0.000000,0.061398,0.000000,0.000000,0.011995
2020-03-04,0.000000,0.000000,0.061398,0.000000,0.000000,0.011995
2020-03-05,0.000000,0.039064,0.184195,0.000000,0.000000,0.047980
2020-03-06,0.000000,0.039064,0.245594,0.000000,0.000000,0.059975
2020-03-07,0.000000,0.117192,0.491187,0.044368,0.000000,0.143940
2020-03-08,0.070512,0.117192,0.491187,0.044368,0.000000,0.155935
2020-03-09,0.141023,0.156256,0.675383,0.088737,0.210021,0.239900
2020-03-10,0.211535,0.390640,1.043773,0.177473,0.420042,0.431819


In [114]:
df2 = cases_per_capita_df
fig2 = go.Figure()
fig2.add_scatter(x=df2.index,y=df2['Bronx County'], mode='lines', name='Bronx')
fig2.add_scatter(x=df2.index,y=df2['Kings County'], mode='lines', name='Brooklyn')
fig2.add_scatter(x=df2.index,y=df2['New York County'], mode='lines', name='Manhattan')
fig2.add_scatter(x=df2.index,y=df2['Queens County'], mode='lines', name='Queens')
fig2.add_scatter(x=df2.index,y=df2['Richmond County'], mode='lines', name='Staten Island')
fig2.add_scatter(x=df2.index,y=df2['Total'], mode='lines+markers', name='NYC')

prepFigure(fig2, title="NYC Cases per Capita (per 100k people)")

In [0]:
fig2.write_html('/content/drive/My Drive/publicweb/nyc-covid19-casespercapita.html', config={"showLink": True,
  "plotlyServerURL": "https://chart-studio.plotly.com"} , include_plotlyjs="cdn")

In [116]:
df = nyc_counties_deaths_df
df = df.loc['2020-03-14':]

deaths_per_capita_df = df.divide(nyc_counties_pop_df['population'], axis=1)*100000

deaths_per_capita_df

County Name,Bronx County,Kings County,New York County,Queens County,Richmond County,Total
Date,,,,,,
2020-03-14,0.000000,0.039064,0.000000,0.000000,0.000000,0.011995
2020-03-15,0.000000,0.039064,0.245594,0.000000,0.000000,0.059975
2020-03-16,0.070512,0.039064,0.000000,0.000000,0.000000,0.023990
2020-03-17,0.070512,0.039064,0.000000,0.000000,0.000000,0.023990
2020-03-18,0.070512,0.039064,0.000000,0.000000,0.000000,0.023990
2020-03-19,0.070512,0.039064,0.000000,0.000000,0.000000,0.023990
2020-03-20,0.070512,0.039064,0.000000,0.000000,0.000000,0.023990
2020-03-21,0.000000,0.039064,0.000000,0.000000,0.000000,0.011995
2020-03-22,0.987162,0.546896,0.613984,0.931736,0.840084,0.755684


In [117]:
df2 = deaths_per_capita_df
fig2 = go.Figure()
fig2.add_scatter(x=df2.index,y=df2['Bronx County'], mode='lines', name='Bronx')
fig2.add_scatter(x=df2.index,y=df2['Kings County'], mode='lines', name='Brooklyn')
fig2.add_scatter(x=df2.index,y=df2['New York County'], mode='lines', name='Manhattan')
fig2.add_scatter(x=df2.index,y=df2['Queens County'], mode='lines', name='Queens')
fig2.add_scatter(x=df2.index,y=df2['Richmond County'], mode='lines', name='Staten Island')
fig2.add_scatter(x=df2.index,y=df2['Total'], mode='lines+markers', name='NYC')

prepFigure(fig2, title="NYC Deaths per Capita (per 100k people)")

In [0]:
fig2.write_html('/content/drive/My Drive/publicweb/nyc-covid19-deathspercapita.html', config={"showLink": True,
  "plotlyServerURL": "https://chart-studio.plotly.com"} , include_plotlyjs="cdn")

# NYC DOHMH Data

## Process Case, Hospitalizations & Deaths Data

NYC's Dept of Health and Mental Hygiene (DOHMH) also publishes a subset of their COVID-19 data...

In [119]:
url = "https://raw.githubusercontent.com/nychealth/coronavirus-data/master/case-hosp-death.csv"

nyc_case_hosp_death_df = pd.read_csv(url)
nyc_case_hosp_death_df.index =pd.to_datetime(nyc_case_hosp_death_df.iloc[:,0])
nyc_case_hosp_death_df = nyc_case_hosp_death_df.drop(nyc_case_hosp_death_df.columns[0], axis=1)

nyc_case_hosp_death_df = nyc_case_hosp_death_df.rename_axis('Date')

nyc_case_hosp_death_df

,NEW_COVID_CASE_COUNT,HOSPITALIZED_CASE_COUNT,DEATH_COUNT
Date,,,
2020-03-02,1,12.0,NaN
2020-03-03,3,7.0,NaN
2020-03-04,10,10.0,NaN
2020-03-05,2,15.0,NaN
2020-03-06,8,11.0,NaN
2020-03-07,14,10.0,NaN
2020-03-08,21,15.0,NaN
2020-03-09,54,35.0,NaN
2020-03-10,75,47.0,NaN


In [120]:
df2 = nyc_case_hosp_death_df
fig2 = go.Figure()
fig2.add_scatter(x=df2.index,y=df2['NEW_COVID_CASE_COUNT'], mode='lines', name='New Cases')
fig2.add_scatter(x=df2.index,y=df2['DEATH_COUNT'], mode='lines', name='Deaths')
fig2.add_scatter(x=df2.index,y=df2['HOSPITALIZED_CASE_COUNT'], mode='lines', name='Hospitalizations', )

prepFigure(fig2,title="NYC Cases/Hospitalizations/Deaths per NYC DOHMH")

fig2.update_layout(annotations=[
            go.layout.Annotation(
                text="""
Per NYC DOHMH:<br><br>
Cases - date of diagnosis<br>
Hospitalizations - date of admission<br>
Deaths - date of death<br>
<br>
Because of delays in reporting, the<br>
most recent data may be incomplete.<br>
Data shown currently will be<br>
updated in the future as new<br>
cases, hospitalizations, and deaths<br>
are reported.
""",
                align='left',
                showarrow=False,
                xref='paper',
                yref='paper',
                x=0.01,
                y=0.96,
                bordercolor='black',
                borderwidth=1
            )
        ] )

In [0]:
fig2.write_html('/content/drive/My Drive/publicweb/nyc-covid19-cases-hosps-deaths.html', config={"showLink": True,
  "plotlyServerURL": "https://chart-studio.plotly.com"} , include_plotlyjs="cdn")

In [122]:
cumsum_df = nyc_case_hosp_death_df.cumsum()
cumsum_df

,NEW_COVID_CASE_COUNT,HOSPITALIZED_CASE_COUNT,DEATH_COUNT
Date,,,
2020-03-02,1,12.0,NaN
2020-03-03,4,19.0,NaN
2020-03-04,14,29.0,NaN
2020-03-05,16,44.0,NaN
2020-03-06,24,55.0,NaN
2020-03-07,38,65.0,NaN
2020-03-08,59,80.0,NaN
2020-03-09,113,115.0,NaN
2020-03-10,188,162.0,NaN


In [123]:
df2 = cumsum_df

fig2 = go.Figure()
fig2.add_scatter(x=df2.index,y=df2['NEW_COVID_CASE_COUNT'], mode='lines', name='New Cases')
fig2.add_scatter(x=df2.index,y=df2['DEATH_COUNT'], mode='lines', name='Deaths')
fig2.add_scatter(x=df2.index,y=df2['HOSPITALIZED_CASE_COUNT'], mode='lines', name='Hospitalizations', )

prepFigure(fig2,title="NYC Cases/Hospitalizations/Deaths per NYC DOHMH")
fig2.update_layout(yaxis_type="log", hovermode="x unified")

In [0]:
fig2.write_html('/content/drive/My Drive/publicweb/nyc-covid19-cum-cases-hosps-deaths.html', config={"showLink": True,
  "plotlyServerURL": "https://chart-studio.plotly.com"} , include_plotlyjs="cdn")

## Process zipcode-level test data

NYC DOHMH also publishes test data down to the zipcode level. However, its not in a time-series format.

Fortunately, since the data is published via GitHub, BetaNYC community member Akil Harris created a utility to help compile the data as a time series by taking advantage of git revisions.

https://github.com/akilism/compile-city-covid

datHere took advantage of this utility and publishes and maintains it in the BetaNYC community portal.

https://data.beta.nyc/dataset/nyc-covid-19-tests-by-zipcode-time-series


In [125]:
url = "https://data.beta.nyc/dataset/14bd6180-de38-4614-a261-bd94820b796f/resource/5e94d9f0-a3fc-4da6-913a-55f1b9c7957f/download/compiled_test_by_zcta.csv"

tests_by_zcta_df = pd.read_csv(url, dtype={'MODZCTA':str})

tests_by_zcta_df['date'] = pd.to_datetime(tests_by_zcta_df['date'])
tests_by_zcta_df['date'] = tests_by_zcta_df['date'].dt.strftime('%Y-%m-%d')
tests_by_zcta_df.rename(columns={'MODZCTA':'zipcode', 'Total':'Tests'}, inplace=True)

df = tests_by_zcta_df.fillna("exclude")
tests_by_zcta_df = df.loc[(df.zipcode != "exclude")]

tests_by_zcta_df

,zipcode,Positive,Tests,date
1,10001,113,265,2020-04-01
2,10002,250,542,2020-04-01
3,10003,161,379,2020-04-01
4,10004,16,38,2020-04-01
5,10005,25,81,2020-04-01
...,...,...,...,...
2846,11691,1459,2203,2020-04-18
2847,11692,376,617,2020-04-18
2848,11693,248,431,2020-04-18
2849,11694,450,797,2020-04-18


There is also a some neighborhood data in the BetaNYC community portal we can use...

https://data.beta.nyc/dataset/pediacities-nyc-neighborhoods

In [126]:
url = "https://data.beta.nyc/dataset/0ff93d2d-90ba-457c-9f7e-39e47bf2ac5f/resource/7caac650-d082-4aea-9f9b-3681d568e8a5/download/nyc_zip_borough_neighborhoods_pop.csv"

neighborhood_df = pd.read_csv(url, dtype={'zip':str})

neighborhood_df

,zip,borough,post_office,neighborhood,population,density
0,10001,Manhattan,"New York, NY",Chelsea and Clinton,21102,33959
1,10002,Manhattan,"New York, NY",Lower East Side,81410,92573
2,10003,Manhattan,"New York, NY",Lower East Side,56024,97188
3,10004,Manhattan,"New York, NY",Lower Manhattan,3089,5519
4,10005,Manhattan,"New York, NY",Lower Manhattan,7135,97048
...,...,...,...,...,...,...
172,11691,Queens,"Far Rockaway, NY",Rockaways,60035,21185
173,11692,Queens,"Arverne, NY",Rockaways,18540,18566
174,11693,Queens,"Far Rockaway, NY",Rockaways,11916,11950
175,11694,Queens,"Rockaway Park, NY",Rockaways,20408,14944


In [127]:
df = pd.merge(tests_by_zcta_df, neighborhood_df, left_on='zipcode', right_on='zip')

df=df.drop(columns=['zip', 'post_office'])

df['tests_percap'] = df['Tests']/df['population']*500
df['positive_percap'] = df['Positive']/df['population']*500 
df['positive_rate'] = df['Positive']/df['Tests'] * 100
df['hover_text'] = df['zipcode'] + ' - ' + df['neighborhood']

df = df.round(2)

df

,zipcode,Positive,Tests,date,borough,neighborhood,population,density,tests_percap,positive_percap,positive_rate,hover_text
0,10001,113,265,2020-04-01,Manhattan,Chelsea and Clinton,21102,33959,6.28,2.68,42.64,10001 - Chelsea and Clinton
1,10001,136,304,2020-04-03,Manhattan,Chelsea and Clinton,21102,33959,7.20,3.22,44.74,10001 - Chelsea and Clinton
2,10001,146,321,2020-04-04,Manhattan,Chelsea and Clinton,21102,33959,7.61,3.46,45.48,10001 - Chelsea and Clinton
3,10001,158,337,2020-04-05,Manhattan,Chelsea and Clinton,21102,33959,7.99,3.74,46.88,10001 - Chelsea and Clinton
4,10001,170,364,2020-04-07,Manhattan,Chelsea and Clinton,21102,33959,8.62,4.03,46.70,10001 - Chelsea and Clinton
...,...,...,...,...,...,...,...,...,...,...,...,...
2828,11697,65,135,2020-04-14,Queens,Rockaways,4079,1838,16.55,7.97,48.15,11697 - Rockaways
2829,11697,69,139,2020-04-15,Queens,Rockaways,4079,1838,17.04,8.46,49.64,11697 - Rockaways
2830,11697,71,143,2020-04-16,Queens,Rockaways,4079,1838,17.53,8.70,49.65,11697 - Rockaways
2831,11697,72,145,2020-04-17,Queens,Rockaways,4079,1838,17.77,8.83,49.66,11697 - Rockaways


# Create Choropleth Maps with extended hyperlocal data

Using [zipcode boundary data from NYU](https://data.beta.nyc/dataset/nyc-zip-code-tabulation-areas), the previously mentioned neighborhood data, along with the derived time-series DOHMH data, we create two geotemporal choropleth maps... one on Positive cases, one for Tests.

In [0]:
from urllib.request import urlopen
import json

with urlopen('https://data.beta.nyc/dataset/3bf5fb73-edb5-4b05-bb29-7c95f4a727fc/resource/5fb3bac2-191f-4b90-a982-e6ece0e96a42/download/nyu-2451-34509-geojson.json') as response:
  nyczcta = json.load(response)


In [0]:
#df = df[df['date'].isin(['2020-04-01', '2020-04-05', '2020-04-07','2020-04-10', '2020-04-13', '2020-04-17'])]

import plotly.express as px

fig = px.choropleth_mapbox(df, geojson=nyczcta, color="Positive",
                           locations="zipcode", featureidkey="properties.zcta",
                           center={"lat":  40.7128, "lon": -74.0060}, animation_frame="date",
                           mapbox_style="stamen-toner", zoom=9.5,
                           title="<b>NYC COVID-19 Positives by zipcode</b>",
                           hover_data=['date', 'Positive', 'Tests', 'population', 'positive_percap', 'tests_percap', 'positive_rate'],
                           hover_name="hover_text",
                           range_color=[0, df['Positive'].max()], height=800)

fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0},
                  annotations=[
            go.layout.Annotation(
                text="""
<b>Legend:</b><br><br>
<b>Positives</b> - positive tests<br>
Tests - number of tests<br>
positive_percap - Positives per 500 people<br>
tests_percap - Tests per 500 people<br>
positive_rate - rate of tests returning positive<br>
""",
                align='left',
                showarrow=False,
                xref='paper',
                yref='paper',
                x=0.01,
                y=0.96,
                bordercolor='black',
                borderwidth=1,
                bgcolor='white'
            )
        ] )

fig.show()

In [0]:
fig.write_html('/content/drive/My Drive/publicweb/nyc-covid19-positives-byzipcode.html', config={"showLink": True,
  "plotlyServerURL": "https://chart-studio.plotly.com"} , include_plotlyjs="cdn")

In [0]:
fig = px.choropleth_mapbox(df, geojson=nyczcta, color="Tests",
                           locations="zipcode", featureidkey="properties.zcta",
                           center={"lat":  40.7128, "lon": -74.0060}, animation_frame="date",
                           mapbox_style="stamen-toner", zoom=9.5,
                           title="<b>NYC COVID-19 Tests by zipcode</b>",
                           hover_data=['date', 'Positive', 'Tests', 'population', 'positive_percap', 'tests_percap', 'positive_rate'],
                           hover_name="hover_text",
                           range_color=[0, df['Tests'].max()], 
                           height=800)

fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0},
                  annotations=[
            go.layout.Annotation(
                text="""
<b>Legend:</b><br><br>
Positives - positive tests<br>
<b>Tests</b> - number of tests<br>
positive_percap - Positives per 500 people<br>
tests_percap - Tests per 500 people<br>
positive_rate - rate of tests returning positive<br>
""",
                align='left',
                showarrow=False,
                xref='paper',
                yref='paper',
                x=0.01,
                y=0.96,
                bordercolor='black',
                borderwidth=1,
                bgcolor='white'
            )
        ] )

fig.show()

In [0]:
fig.write_html('/content/drive/My Drive/publicweb/nyc-covid19-tests-byzipcode.html', config={"showLink": True,
  "plotlyServerURL": "https://chart-studio.plotly.com"} , include_plotlyjs="cdn")